互评作业三： 基于 K-means 的文本聚类

任务：本作业将通过 K-means 算法对 20 Newsgroups 数据集中的文本数据进行聚类。首先是数据的预处理，提取文本特征；然后将数据转换为特征向量，使用 K-means 算法进行聚类；最后对聚类结果进行评估和可视化分析。

数据集：20 Newsgroups

1. 数据预处理：去除停用词、数字、符号等，提取文本特征。

In [14]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import re

# 加载数据集，20 Newsgroups 可以直接从fetch_20newsgroups中加载
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

# 去除停用词、数字、符号等
def preprocessor(text):
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    return text

2. 特征向量化：将预处理后的文本使用 TF-IDF 方法进行向量化。

In [15]:
#使用TF-IDF进行向量化
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(newsgroups_train.data)
print(X.shape)
print(X.nnz / float(X.shape[0]))

(11314, 101322)
66.802987449178


从上述输出结果可看出，提取的TF-IDF 向量有66个非零特征。

3. K-means 模型训练：根据设定的 K 值，构建 K-means 模型，并对向量化的文本数据进行聚类。

In [6]:
import numpy as np
from sklearn.cluster import KMeans

# 建立文本聚类模型
k = 20
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1, verbose=0)

# 训练模型
kmeans = model.fit(X)

# 聚类结果展示
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

for i in range(k):
    print("Cluster %d:" % (i+1))
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])

4. 聚类结果分析：评估聚类结果，可使用轮廓系数、CH 指数等方法。

In [17]:
#计算轮廓系数分析聚类结果
from sklearn.metrics import silhouette_score
lables = kmeans.fit_predict(X)
score = silhouette_score(X, lables)
print("Silhouette Coefficient: %0.3f" % score)

Silhouette Coefficient: 0.902


轮廓系数是一种用于评估聚类结果的指标。它可以衡量聚类结果的紧密度和分离度，值越接近1表示聚类效果越好,上述结果表示当前聚类结果的轮廓系数为0.902，聚类效果较好。

5. 可视化：使用降维方法（如 PCA 或 t-SNE）将高维数据降维至 2 D 或 3 D，然后进行可视化，观察聚类效果。

In [1]:
#使用PCA降维对数据处理，将高维数据降低至二维
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X.toarray())
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels)
plt.show()